In [1]:
docs = [
  '먹고 싶은 사과', # 문서0 
  '먹고 싶은 바나나', # 문서1
  '길고 노란 바나나 바나나', # 문서2 
  '저는 과일이 좋아요' # 문서3 
]

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

In [3]:
# 문장을 counter vectorizer 형태로 변형해주기
countvect = vect.fit_transform(docs)
countvect

<4x9 sparse matrix of type '<class 'numpy.int64'>'
	with 12 stored elements in Compressed Sparse Row format>

toarray()를 통해서 문장이 vector 형태의 값을 얻을 수 있음(np 형태의 값으로 바꿔줌)

하지만 각 인덱스와 칼럼이 무엇을 의미하는지에 대해서는 알 수 없음

In [5]:
countvect.toarray()

array([[0, 0, 0, 1, 0, 1, 1, 0, 0],
       [0, 0, 0, 1, 1, 0, 1, 0, 0],
       [0, 1, 1, 0, 2, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 1, 1]], dtype=int64)

단어사전 확인

In [6]:
vect.vocabulary_

{'먹고': 3,
 '싶은': 6,
 '사과': 5,
 '바나나': 4,
 '길고': 1,
 '노란': 2,
 '저는': 7,
 '과일이': 0,
 '좋아요': 8}

In [13]:
# 슌서가 뒤죽박죽이니까 정렬을 시켜주자
sorted(vect.vocabulary_)

['과일이', '길고', '노란', '먹고', '바나나', '사과', '싶은', '저는', '좋아요']

In [8]:
import pandas as pd
countvect_df = pd.DataFrame(countvect.toarray(), columns = sorted(vect.vocabulary_))
countvect_df.index = ['문서1', '문서2', '문서3', '문서4']
countvect_df

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
문서1,0,0,0,1,0,1,1,0,0
문서2,0,0,0,1,1,0,1,0,0
문서3,0,1,1,0,2,0,0,0,0
문서4,1,0,0,0,0,0,0,1,1


위의 데이터 프레임 형태의 유사도를 계산

In [10]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(countvect_df,countvect_df)

array([[1.        , 0.66666667, 0.        , 0.        ],
       [0.66666667, 1.        , 0.47140452, 0.        ],
       [0.        , 0.47140452, 1.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        ]])

#### 이번에는 TF-IDF로 해보자

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()
tfvect = vect.fit(docs)
tfvect

TfidfVectorizer()

In [12]:
tfidf_df = pd.DataFrame(tfvect.transform(docs).toarray(), columns = sorted(vect.vocabulary_))
tfidf_df.index = ['문서1', '문서2', '문서3', '문서4']
tfidf_df

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
문서1,0.00000,0.00000,0.00000,0.526405,0.00000,0.667679,0.526405,0.00000,0.00000
문서2,0.00000,0.00000,0.00000,0.577350,0.57735,0.000000,0.577350,0.00000,0.00000
문서3,0.00000,0.47212,0.47212,0.000000,0.74445,0.000000,0.000000,0.00000,0.00000
문서4,0.57735,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.57735,0.57735


-----------

### 위키독스 예재

In [14]:
import os

In [15]:
print(os.listdir('./data/movie-kaggle/'))

['credits.csv', 'keywords.csv', 'links.csv', 'links_small.csv', 'movies_metadata.csv', 'ratings.csv', 'ratings_small.csv']


In [16]:
path = './data/movie-kaggle/'

In [17]:
data = pd.read_csv(path + 'movies_metadata.csv', low_memory = False)
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [18]:
# over view의 항목 추출
data.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

전처리

In [19]:
data['overview']

0        Led by Woody, Andy's toys live happily in his ...
1        When siblings Judy and Peter discover an encha...
2        A family wedding reignites the ancient feud be...
3        Cheated on, mistreated and stepped on, the wom...
4        Just when George Banks has recovered from his ...
                               ...                        
45461          Rising and falling between a man and woman.
45462    An artist struggles to finish his work while a...
45463    When one of her hits goes wrong, a professiona...
45464    In a small town live two brothers, one a minis...
45465    50 years after decriminalisation of homosexual...
Name: overview, Length: 45466, dtype: object

In [22]:
# overview에 있는 결측치가 있는 항목들을 제거해주자
# 결측치가 없으면 True 있으면 False라고 나오게 된다
# True인것만 남기자!
data = data[data['overview'].notnull()].reset_index(drop=True)
data.shape

(44512, 24)

In [28]:
# 전체 데이터 용량이 많으니깐 2만개 데이터만 해서 해보자
data = data.loc[0:19999].reset_index(drop = True)
data.shape

(20000, 24)

In [29]:
# 불용어 처리를 해주자
tfidf = TfidfVectorizer(stop_words = 'english')

# overview에 대해서 tf-idf  수행
tfidf_matrix = tfidf.fit_transform(data['overview'])
print(tfidf_matrix.shape)

(20000, 47661)


In [31]:
# 코사인 유사도 계산해보기!
cosine_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [32]:
cosine_matrix.shape

(20000, 20000)

In [34]:
import numpy as np

np.round(cosine_matrix, 4)

array([[1.    , 0.0158, 0.    , ..., 0.    , 0.0083, 0.0172],
       [0.0158, 1.    , 0.0492, ..., 0.0149, 0.0057, 0.008 ],
       [0.    , 0.0492, 1.    , ..., 0.    , 0.    , 0.    ],
       ...,
       [0.    , 0.0149, 0.    , ..., 1.    , 0.0027, 0.    ],
       [0.0083, 0.0057, 0.    , ..., 0.0027, 1.    , 0.0144],
       [0.0172, 0.008 , 0.    , ..., 0.    , 0.0144, 1.    ]])

In [35]:
# cosine matrix 안에 인덱스가 20000까지 있는데
# 0번 인덱스 : 0번째 id 영화

In [ ]:
# movie title과 id를 dictionary로 생성해준다
mnovie2id = {}

for i,c enumerate(data['title']) : movie2id[i] = c